### Basic import

In [ ]:
import rosbag
import roslib
import cv2
import numpy as np
import pandas as pd
from cv_bridge import CvBridge, CvBridgeError
from rospy import Subscriber
from sensor_msgs.msg import Image
from opt_msgs.msg import DetectionArray
import message_filters
from message_filters import TimeSynchronizer, Subscriber
import socket, json
import re

In [ ]:
# Create bag to process
bag = rosbag.Bag('../../bags/quality.bag')
print bag

In [ ]:
# create bridge between CV and ROS
br = CvBridge()

In [ ]:
# Create array for rgb frames
rgbTime = []
rgbSeq = []
for topic, msg, t in bag.read_messages(topics=['/kinect2_head/rgb_lowres/image']):
    time = msg.header.stamp
    rgbTime.append(int(str(time)))
    seq = msg.header.seq
    rgbSeq.append(seq)
    msg.encoding = "bgr8"
    cv_image = br.imgmsg_to_cv2(msg, desired_encoding="bgr8")
    cv2.imwrite('/home/sabrine/notebook/framesRGB/vajm' + str(seq) + '.jpg',cv_image)

In [ ]:
len(rgbTime)

In [ ]:
# Create a txt file with detection msg separate by 2 new lines
file = open("/home/sabrine/notebook/messages/detectQuality.txt","w")

# Creation of empty list thta will be fill by extracted information and then use to create a dataframe of useful info
detectTime = []
detectSeq = []
box2dList = []
heightList = []
distanceList = []
i = 0

for topic, msg, t in bag.read_messages(topics=['/detector/detections']):
    #print (str(msg).split('\n'))
    #break
    #for e in str(msg).split('\n'):
        #if 'Laurene' in e:
        
    # Extract the detection field that start and end with a regular pattern 
    begin = str(msg).find('detections:')
    end = str(msg).find('intrinsic_matrix:')
    if i < 20:
        detections = str(msg)[begin:end]
        
        # Extract the several detection for this msg
        detection = detections.split("- ")
        #A non empty detection will be split in more than 1 cell
        if len(detection) > 1:
            for i in range(1, len(detection)):
                
                # Extract box2d coordonates
                b = detection[i].find('box_2D:')
                e = detection[i].find('centroid:')
                box2D = detection[i][b:e]
                box2dList.append(box2D)
                
                # Reduce information to avoid ambiguious match
                newdetection = detection[i][e:]
                
                # Extract height of the detection
                b = newdetection.find('height:')
                e = newdetection.find('confidence:')
                h = re.search('\d+\.\d+',newdetection[b:e]).group(0)
                heightList.append(float(h))
                
                # Reduce information to avoid ambiguious match
                newdetectionre = newdetection[e:]
                
                # Extract distance of the detection
                b = newdetectionre.find('distance:')
                e = newdetectionre.find('occluded:')
                d = re.search('\d+\.\d+',newdetectionre[b:e]).group(0)
                distanceList.append(float(d))
                
                # Extract timestamp
                time = msg.header.stamp
                detectTime.append(int(str(time)))
                
                #Extract seq number
                seq = msg.header.seq
                detectSeq.append(seq)
    file.write(str(msg) + "\n \n")  
    
    i = i + 1

file.close()

In [ ]:
len(distanceList)

In [ ]:
rgb = pd.DataFrame(data=rgbTime, index=rgbSeq)

In [ ]:
rgb['seq'] = rgb.index

In [ ]:
detect = pd.DataFrame(data=detectTime, index=detectSeq)

In [ ]:
def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return array[idx]

In [ ]:
match_frame= []
for index, row in detect.iterrows():
    #print index
    #print row[0]
    rgbSeqMatched = find_nearest(np.asarray(rgbTime), row[0])
    match_frame.append(rgbSeqMatched)

In [ ]:
detect['match_frame'] = match_frame

In [ ]:
detect['seq'] = detect.index

In [ ]:
synchronize = pd.merge(detect, rgb, how='inner', left_on='match_frame', right_on=0,
         left_index=False, right_index=False, sort=True,
         suffixes=('_msg', '_rgb'), copy=True, indicator=False)

In [ ]:
synchronize

In [ ]:
synchronize['seq_rgb']

In [ ]:
synchronize.index = synchronize['seq_msg']

In [ ]:
synchronize['seq_rgb'][3972]

In [ ]:
np.save('rgbDetectionSync', synchronize['seq_rgb'])